In [85]:
url = 'https://github.com'

## @マークが入っているか

In [86]:
def at_sign_symdol(url_data):
    for character in url_data:
        if(character == "@"):
            ans = -1
            break
        else:
            ans = 0
    return ans

## どこにも行かないlinkがあるか

In [168]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def url_of_anchor(url_data):
    url = url_data
    parsed_url = urlparse(url)
    headers = {'User-Agent': 'Mozilla/5.0'}

    response = requests.get(url, headers=headers) # <Response [200]>

    soup = BeautifulSoup(response.text, 'html.parser')
    
    cnt = 0.01
    anc_cnt = 0.0
    for link in soup.findAll('a', href=True):
        cnt += 1.0
        if "#" == link['href']:
            anc_cnt += 1.0
        elif "#content" == link['href']:
            anc_cnt += 1.0
        elif "#skip" == link['href']:
            anc_cnt += 1.0
        elif "JavaScript::void(0)" in link['href']:
            anc_cnt += 1.0

    per = anc_cnt / cnt*100
    if per < 31:
        return 1
    elif 31 <= per and per >= 67:
        return 0
    else:
        return -1

## URLの長さ

In [141]:
def long_url(url_data):
    url_len = len(url_data)
    if 75 >= url_len >= 54:
        ans = 0
    elif url_len >= 54:
        ans = -1
    elif url_len < 54:
        ans = 1
    return ans

## Emailを送信させる機能

In [154]:
def is_send_email(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers) # <Response [200]>
    soup = BeautifulSoup(response.text, 'html.parser')

    
    cnt = 0.0
    anc_cnt = 0.0
    for link in soup.findAll('a', href=True):
        cnt += 1.0
        if "mailto" in link['href']:
            return -1
        elif "mail()" in link['href']:
            return -1
    return 1


# システム

## 実装部分

In [169]:
url = 'https://support.apple.com-a2fszw0gymfuz3nhdcbldxl5.store/'

url_date = []

url_date.append(at_sign_symdol(url))
url_date.append(url_of_anchor(url))
url_date.append(long_url(url))
url_date.append(is_send_email(url))

pred = clf.predict_proba(url_date) 

fishing = pred.tolist()
print("フィッシングサイトの確率は{0:.1f}%".format(fishing[0][0]*100))

フィッシングサイトの確率は20.2%


/Users/kajyuuen/.anyenv/envs/pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


## モデル構築部分

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [157]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("df.csv")
train_df, test_df = train_test_split(df[["having_At_Symbol","URL_Length","URL_of_Anchor","Submitting_to_email", "Result"]], test_size = 0.2)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

clf = LogisticRegression(C=1, penalty='l2')
clf.fit(train_df.drop('Result', axis=1), train_df['Result'])
pred = clf.predict(test_df.drop('Result', axis=1))
# pred = clf.predict_proba(test_df.drop('Result', axis=1)) 
print('{:.5f}'.format(accuracy_score(test_df['Result'], pred)))

0.84939
